In [0]:
%run "./config"

In [0]:
df_accounts = spark.read.format("parquet").load(f"{silver_folder_path}/accounts/*.parquet")

In [0]:
df_customers=spark.read.format("parquet").load(f"{silver_folder_path}/customers/*.parquet")

In [0]:
df_loan_payments=spark.read.format("parquet").load(f"{silver_folder_path}/loan_payments/*.parquet")

In [0]:
df_loans=spark.read.format("parquet").load(f"{silver_folder_path}/loans/*.parquet")

In [0]:
df_transactions=spark.read.format("parquet").load(f"{silver_folder_path}/transactions/*.parquet")

In [0]:
# join accounts and customers
df_accounts_customers = df_accounts.alias("a"). \
    join(df_customers.alias("c"), "customer_id"). \
    select("a.account_id", "a.customer_id", "c.first_name", "c.last_name")

In [0]:
#join loan and loan payments
df_loan_loanppayments = df_loan_payments.alias("lp"). \
    join(df_loans.alias("l"), "loan_id"). \
        select("lp.*", "l.customer_id","l.loan_amount")
            

In [0]:
#join df_loan_loanppayments with df_accounts_customers
df_llp_ac=df_loan_loanppayments.alias("llp").\
    join(df_accounts_customers.alias("ac"), "customer_id").\
        select("llp.*", "ac.account_id")

In [0]:
#join transactions with df_llp_ac
df_joined=df_transactions.alias("t"). \
    join(df_llp_ac.alias("llp"), "account_id"). \
        select("t.*", "llp.loan_id", "llp.payment_id","llp.payment_date","llp.payment_amount","llp.loan_amount")

In [0]:
#write to silver in parquet
df_joined.write.mode("overwrite").format("parquet").save(f"{silver_folder_path}/joined")